In [4]:
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV

# Data Preprocessing

In [9]:
nba_player_data_2025_26 = pd.read_csv('out/player_stats_2025-26.csv')
nba_player_data_2024_25 = pd.read_csv('out/player_stats_2024-25.csv')
nba_players = pd.read_csv('out/players.csv')
nba_teams = pd.read_csv('out/teams.csv')
nba_season_games_2024_25 = pd.read_csv('out/season_games_2024-25.csv')
nba_season_games_2025_26 = pd.read_csv('out/season_games_2025-26.csv')


In [ ]:
opt = BayesSearchCV(
    XGBClassifier(),
    {
        'n_estimators': (10, 100),
        'max_depth': (3, 10),
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'subsample': (0.5, 1.0),
        'colsample_bytree': (0.5, 1.0),
    },
    n_iter=32,
    cv=3
)

In [ ]:
data_year_24_25 = pd.read_csv("player_stats_2024_25.csv")
data_year_25_26 = pd.read_csv("player_stats_2025_26.csv")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size=.2)
# create model instanc
model = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
# fit model
model.fit(X_train, y_train)
# make predictions
preds = model.predict(X_test)